In [1]:
# Standard modules
from dataclasses import dataclass
from logging import getLogger, StreamHandler, DEBUG, Formatter

# Third party modules
import gffutils
import pybedtools
import pandas as pd
import numpy as np
from pathlib2 import Path
from tqdm import tqdm

# Local modules
from libs.args import parser_setting
from libs.utils import load_config, OutputSettings
# from libs.preprocess import PreProcessExomeSummary
# from libs.modesamples import ModeSamples
# from libs.annolibs.anno import Anno
# from libs.filter.maffilter import MafFilter
# from libs.filter.typefilter import TypeFilter
# from libs.filter.gtfilter import GtFilter
# from libs.filter.counter import counter

# Settings
tqdm.pandas()
logger = getLogger(__name__)
handler = StreamHandler()
handler.setLevel(DEBUG)
logger.setLevel(DEBUG)
logger.addHandler(handler)

In [9]:
args = parser_setting()
# dgvfile = f"/work{args['resources']}/DGV/DGV.GS.March2016.50percent.GainLossSep.Final.hg19.gff3"
dgvfile = '/Volumes/resources/DGV/DGV.GS.March2016.50percent.GainLossSep.Final.hg19.gff3'

In [14]:
# xhmm = pd.read_table(args['xhmm'], sep='\t', header=0)
xhmm = pd.read_table('/Volumes/vol/work/Github/TestData/proband/xhmm/data.segdup.strvar.haplo.deciph.omim.xcnv.gene.uniq', sep='\t', header=0)
dgv = pd.read_table(dgvfile, sep='\t', header=None)


In [15]:
xhmm

,SAMPLE,CNV,INTERVAL,KB,CHR,MID_BP,TARGETS,NUM_TARG,Q_EXACT,Q_SOME,...,Q_START,Q_STOP,MEAN_RD,MEAN_ORIG_RD,SegDup,StrVar,Haplo_Insuff_Trip,Decipher,Omimgene,gene
0,Sample_33441,DUP,13:114437940-114524091,86.15,13,114481015,130620..130633,14,9,95,...,8,9,2.86,77.77,0.000000,80.792088,0.0,0.0,0.0,"TMEM255B,LINC00552,GAS6,GRK1"
1,Sample_33638,DEL,1:104162075-104297343,135.27,1,104229709,9162..9166,5,5,98,...,5,49,-4.42,23.80,99.611884,100.000000,0.0,0.0,100.0,"AMY2A,AMY1A,AMY1C,AMY1B"
2,Sample_33613,DUP,14:106372968-106380348,7.38,14,106376658,136877..136880,4,23,65,...,12,23,4.46,70.24,42.338437,100.000000,0.0,0.0,0.0,-
3,Sample_33607,DEL,5:17604789-17611792,7.00,5,17608290,55111..55112,2,31,31,...,31,31,-5.95,31.25,100.000000,100.000000,0.0,0.0,100.0,-
4,Sample_33042,DUP,3:100009321-100042612,33.29,3,100025966,40169..40182,14,12,97,...,25,12,3.49,82.96,0.000000,0.000000,0.0,0.0,0.0,TBC1D23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3099,Sample_33390,DUP,16:55844328-55861894,17.57,16,55853111,148082..148092,11,38,97,...,4,26,3.60,87.17,100.000000,100.000000,0.0,0.0,0.0,CES1
3100,Sample_33395,DUP,12:10583615-10594999,11.38,12,10589307,117293..117301,9,91,94,...,56,44,7.22,94.11,100.000000,100.000000,0.0,0.0,100.0,"KLRC2,KLRC1"
3101,Sample_33392,DEL,5:730448-824775,94.33,5,777611,54398..54406,9,12,41,...,8,12,-2.67,45.85,95.476423,100.000000,0.0,100.0,0.0,"ZDHHC11,ZDHHC11B"
3102,Sample_32412,DEL,16:28617041-28621612,4.57,16,28619326,146885..146889,5,13,65,...,13,22,-3.85,56.60,100.000000,100.000000,0.0,100.0,100.0,SULT1A1


In [16]:
proband_id = 'Sample_33727'
xhmm = xhmm[xhmm['SAMPLE'] == proband_id]

In [20]:
result = []
gff_file = pybedtools.BedTool(dgvfile)

In [37]:
for index, row in tqdm(xhmm.iterrows(), total=len(xhmm)):
    print(row['INTERVAL'], type(row['INTERVAL']))

100%|██████████| 23/23 [00:00<00:00, 11236.92it/s]

22:20376922-20484649 <class 'str'>
7:5959379-6006847 <class 'str'>
7:6797208-6845768 <class 'str'>
1:1634814-1671096 <class 'str'>
13:114514608-114537745 <class 'str'>
1:16350194-16360253 <class 'str'>
7:100331691-100336336 <class 'str'>
11:89499659-89606558 <class 'str'>
19:54735516-54755132 <class 'str'>
22:24277563-24331509 <class 'str'>
17:39661207-39684599 <class 'str'>
5:716822-766150 <class 'str'>
19:58131427-58153649 <class 'str'>
14:20212798-20404861 <class 'str'>
1:16383581-16386199 <class 'str'>
7:141782174-141785861 <class 'str'>
1:149379059-149400463 <class 'str'>
14:106050390-106173999 <class 'str'>
1:25598938-25655727 <class 'str'>
15:22742295-22743698 <class 'str'>
1:47608840-47614569 <class 'str'>
16:69988136-70208405 <class 'str'>
21:10862521-10944887 <class 'str'>


In [39]:
re.split('[:-]', '21:10862521-10944887')

['21', '10862521', '10944887']

In [40]:
import re

for index, row in tqdm(xhmm.iterrows(), total=len(xhmm)):
    position: list = re.split('[:-]', row['INTERVAL'])
    chrom, start, end = position[0], int(position[1]) - 1, int(position[2])

    interval = pybedtools.Interval(chrom, start, end)
    if gff_file.any_hits(interval):
        result.append(True)
    else:
        result.append(False)


100%|██████████| 23/23 [01:11<00:00,  3.11s/it]


In [41]:
xhmm.loc[:,'is_in_DGV'] = result

/var/folders/b_/1qjd6n853xbbctck9_20_nt40000gn/T/ipykernel_9376/418515107.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xhmm['is_in_DGV'] = result


In [42]:
xhmm

,SAMPLE,CNV,INTERVAL,KB,CHR,MID_BP,TARGETS,NUM_TARG,Q_EXACT,Q_SOME,...,Q_STOP,MEAN_RD,MEAN_ORIG_RD,SegDup,StrVar,Haplo_Insuff_Trip,Decipher,Omimgene,gene,is_in_DGV
41,Sample_33727,DUP,22:20376922-20484649,107.73,22,20430785,182045..182049,5,45,99,...,41,8.03,75.86,100.000000,100.0,0.0,100.0,100.000000,"TMEM191B,PI4KAP1,RIMBP3",False
122,Sample_33727,DEL,7:5959379-6006847,47.47,7,5983113,73740..73748,9,7,78,...,7,-2.98,23.45,100.000000,100.0,0.0,0.0,0.000000,"RSPH10B2,RSPH10B,CCZ1",False
251,Sample_33727,DUP,7:6797208-6845768,48.56,7,6821488,73891..73901,11,4,94,...,4,2.95,33.11,100.000000,100.0,0.0,0.0,0.000000,"RSPH10B2,RSPH10B,CCZ1B",False
395,Sample_33727,DUP,1:1634814-1671096,36.28,1,1652955,264..277,14,15,46,...,16,2.28,54.51,78.309401,100.0,0.0,100.0,0.000000,"SLC35E2A,CDK11A,CDK11B",False
492,Sample_33727,DUP,13:114514608-114537745,23.14,13,114526176,130632..130639,8,58,95,...,37,6.37,76.78,0.000000,100.0,0.0,0.0,0.000000,"TMEM255B,GAS6",False
590,Sample_33727,DUP,1:16350194-16360253,10.06,1,16355223,1922..1935,14,8,99,...,10,2.94,55.45,100.000000,100.0,0.0,0.0,100.000000,CLCNKA,False
636,Sample_33727,DEL,7:100331691-100336336,4.65,7,100334013,78565..78568,4,44,93,...,17,-5.62,0.00,0.000000,100.0,0.0,0.0,0.000000,ZAN,False
1194,Sample_33727,DEL,11:89499659-89606558,106.90,11,89553108,112824..112830,7,9,51,...,14,-3.06,18.59,100.000000,100.0,0.0,0.0,0.000000,"TRIM49,TRIM64B,TRIM53AP,TRIM51EP",False
1288,Sample_33727,DUP,19:54735516-54755132,19.62,19,54745324,173945..173949,5,5,96,...,6,4.39,80.25,26.874650,100.0,0.0,0.0,0.000000,"LILRA6,LILRB5,LILRB3",False
1373,Sample_33727,DUP,22:24277563-24331509,53.95,22,24304536,182472..182481,10,4,99,...,4,3.81,49.68,95.877435,100.0,0.0,0.0,100.000000,"GSTT2B,DDT,GSTT2,DDTL",False


In [17]:
xhmm

,SAMPLE,CNV,INTERVAL,KB,CHR,MID_BP,TARGETS,NUM_TARG,Q_EXACT,Q_SOME,...,Q_START,Q_STOP,MEAN_RD,MEAN_ORIG_RD,SegDup,StrVar,Haplo_Insuff_Trip,Decipher,Omimgene,gene
41,Sample_33727,DUP,22:20376922-20484649,107.73,22,20430785,182045..182049,5,45,99,...,4,41,8.03,75.86,100.000000,100.0,0.0,100.0,100.000000,"TMEM191B,PI4KAP1,RIMBP3"
122,Sample_33727,DEL,7:5959379-6006847,47.47,7,5983113,73740..73748,9,7,78,...,19,7,-2.98,23.45,100.000000,100.0,0.0,0.0,0.000000,"RSPH10B2,RSPH10B,CCZ1"
251,Sample_33727,DUP,7:6797208-6845768,48.56,7,6821488,73891..73901,11,4,94,...,14,4,2.95,33.11,100.000000,100.0,0.0,0.0,0.000000,"RSPH10B2,RSPH10B,CCZ1B"
395,Sample_33727,DUP,1:1634814-1671096,36.28,1,1652955,264..277,14,15,46,...,15,16,2.28,54.51,78.309401,100.0,0.0,100.0,0.000000,"SLC35E2A,CDK11A,CDK11B"
492,Sample_33727,DUP,13:114514608-114537745,23.14,13,114526176,130632..130639,8,58,95,...,14,37,6.37,76.78,0.000000,100.0,0.0,0.0,0.000000,"TMEM255B,GAS6"
590,Sample_33727,DUP,1:16350194-16360253,10.06,1,16355223,1922..1935,14,8,99,...,4,10,2.94,55.45,100.000000,100.0,0.0,0.0,100.000000,CLCNKA
636,Sample_33727,DEL,7:100331691-100336336,4.65,7,100334013,78565..78568,4,44,93,...,12,17,-5.62,0.00,0.000000,100.0,0.0,0.0,0.000000,ZAN
1194,Sample_33727,DEL,11:89499659-89606558,106.90,11,89553108,112824..112830,7,9,51,...,10,14,-3.06,18.59,100.000000,100.0,0.0,0.0,0.000000,"TRIM49,TRIM64B,TRIM53AP,TRIM51EP"
1288,Sample_33727,DUP,19:54735516-54755132,19.62,19,54745324,173945..173949,5,5,96,...,10,6,4.39,80.25,26.874650,100.0,0.0,0.0,0.000000,"LILRA6,LILRB5,LILRB3"
1373,Sample_33727,DUP,22:24277563-24331509,53.95,22,24304536,182472..182481,10,4,99,...,39,4,3.81,49.68,95.877435,100.0,0.0,0.0,100.000000,"GSTT2B,DDT,GSTT2,DDTL"
